In [1]:
import numpy as np
import gdal
import glob
import os
import datetime
from scipy.interpolate import griddata
import re
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [6]:
# Collect Files
file_dict = {
    'EVI' : glob.glob('MODIS_NDVI/MOD13*EVI*.tif'),
    'EVI_QC' : glob.glob('MODIS_NDVI/*pixel_reliability*.tif'),
    'TA' : glob.glob('MODIS_TA/MOD14*FireMask*.tif'),
    'TA_QC' : glob.glob('MODIS_TA/*QA*.tif'),
    'LST_Day' : glob.glob('MODIS_LST_Day/*LST_Day*.tif'),
    'LST_Day_QC' : glob.glob('MODIS_LST_Day/*QC_Day*.tif'),
    'LST_Night' : glob.glob('MODIS_LST_Night/*LST_Night*.tif'),
    'LST_Night_QC' : glob.glob('MODIS_LST_Night/*QC_Night*.tif')
}

In [3]:
# Open files and store as netcdfs
def open_xarray(data, file_dict):
    """
    Opens dataset as an xarray DataArray.
    
    :param data: key from file_dict
    :param file_dict: dictionary of file paths
    
    :returns: xarray data structure"""
    v = file_dict[data]
    pat = r'doy(\d{7})'
    da_list = []
    for fp in v:
        time = re.findall(pat, fp)[0]
        year = int(time[:4])
        day = int(time[-3:])
        da = xr.open_rasterio(fp)
        dt = (datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1))
        da = da.assign_coords(time=dt)
        da_list.append(da)
    da = xr.concat(da_list, dim='time').squeeze()
    da.name = data
    da.values = da.values * eval(da.attrs['scale_factor'])  # apply scale factor 
    
    return da


In [15]:
ta_qc_da = open_xarray('TA_QC', file_dict)

In [16]:
np.unique(ta_qc_da.values, return_counts=True)

(array([-999,    0,    1,    2,    4,    5,    6], dtype=int16),
 array([46558717,   380106,    16991,  3420767,   697882,    37598,
        18794141]))

In [4]:
evi_qc_da = open_xarray('EVI_QC', file_dict)

In [7]:
_, num = np.unique(evi_qc_da, return_counts=True)
np.unique(evi_qc_da, return_counts=True)

(array([-1,  0,  1,  2,  3], dtype=int32),
 array([24235671,  9317419,  1245754,   625456,   272484]))

In [20]:
lst_day_qc_da = open_xarray('LST_Day_QC', file_dict)

In [21]:
np.unique(lst_day_qc_da, return_counts=True)

(array([  0,   2,   3,  17,  33,  49,  65,  81,  97, 113, 129, 145, 177],
       dtype=uint8),
 array([59021958,   438224,   893141,  4281835,    46662,       34,
         2583384,  2497094,    64028,       81,     1248,    78511,
               2]))

In [22]:
lst_night_qc_da = open_xarray('LST_Night_QC', file_dict)

In [23]:
np.unique(lst_night_qc_da, return_counts=True)

(array([  0,   2,   3,  17,  33,  49,  65,  81,  97, 113, 129, 145, 161,
        177], dtype=uint8),
 array([58833409,   378544,   893141,  4789980,    12255,       39,
         3178346,  1707528,    23995,      137,     2524,    86170,
             132,        2]))

In [8]:
def interp_da(to_interp, interp_like):
    """
    Function intended to perform temporal interpolation on the EVI data.
    
    :param to_interp: the DataArray to temporally interpolate
    :param interp_like: the DataArray with the desired temporal resolution
    
    :returns: interpolated input DA
    """
    return to_interp.interp({'time':interp_like.time}, method='linear')

In [50]:
evi = open_xarray('EVI', file_dict)

In [51]:
ta = open_xarray('TA', file_dict)

In [52]:
evi = interp_da(evi, ta)

In [7]:
file_dict['LST_Day']

['MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018297_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2019121_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2019033_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2019001_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2019113_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2019089_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018241_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018353_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018361_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018273_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018209_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018257_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018345_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018329_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.006_LST_Day_1km_doy2018265_aid0001.tif',
 'MODIS_LST_Day/MOD11A2.0